# VacationPy

In [1]:
# -- dependencies and setup ---
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json



# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# --- store the filepath in a variable ---
file = "../output_data/cities.csv"

# --- read the csv file and create a dataframe ---
cities_data = pd.DataFrame(pd.read_csv(file, index_col="City ID"))

# --- display the dataframe ---
cities_data

,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
City ID,,,,,,,,,
0,Necochea,AR,2022-10-17 05:46:46,-38.5473,-58.7368,45.28,90,79,7.90
1,Tuktoyaktuk,CA,2022-10-17 05:46:46,69.4541,-133.0374,28.40,93,100,20.71
2,Petropavlovsk-Kamchatskiy,RU,2022-10-17 05:46:46,53.0452,158.6483,39.79,60,75,4.47
3,Sola,VU,2022-10-17 05:46:46,-13.8833,167.5500,81.16,76,44,15.95
4,Blagoyevo,RU,2022-10-17 05:46:46,63.3667,47.9167,41.59,92,93,9.71
...,...,...,...,...,...,...,...,...,...
578,Mount Isa,AU,2022-10-17 05:48:44,-20.7333,139.5000,92.97,24,40,5.75
579,Lerwick,GB,2022-10-17 05:48:45,60.1545,-1.1494,50.00,74,66,12.24
580,Tessalit,ML,2022-10-17 05:48:45,20.1986,1.0114,77.85,20,8,12.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# --- access maps with unique API key ---
gmaps.configure(api_key=g_key)

# --- use Lat and Lng as locations and Humidity as the weight ---
locations = cities_data[["Lat", "Lng"]]
weights = cities_data["Humidity (%)"]

# --- customize the size of the figure and add heatmap layer to map ---
figure_layout = {
    'width': "100%",
    'height': '400px',
#    'border': '1px solid black',
    'padding': '1px',
#    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center=(20.0, 5.0), zoom_level=1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False)
fig.add_layer(heatmap_layer)

# --- set maximum intensity and point_radius settings ---
heatmap_layer.max_intensity = max(weights)
heatmap_layer.point_radius = 3

# --- display heatmap ---
fig

Figure(layout=FigureLayout(height='400px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# --- create dataframe with conditional statements to fit the below criteria ---
# 1. A max temperature lower than 80 degrees but higher than 70.
# 2. Wind speed less than 10 mph.
# 3. Zero cloudiness.
# 4. Humidity is less than 75%
# 5

ideal_cities = cities_data.loc[(cities_data["Max Temp (F)"] > 70) & 
                               (cities_data["Max Temp (F)"] < 80) & 
                               (cities_data["Wind Speed (mph)"] < 10) & 
                               (cities_data["Cloudiness (%)"] == 0) &
                               (cities_data["Humidity (%)"] < 75), :].reset_index(drop=True).dropna()
ideal_cities

,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Makakilo City,US,2022-10-17 05:46:49,21.3469,-158.0858,79.12,68,0,6.91
1,Pahrump,US,2022-10-17 05:46:54,36.2083,-115.9839,71.26,41,0,1.01
2,Antalaha,MG,2022-10-17 05:46:59,-14.9003,50.2788,77.25,52,0,7.36
3,Agadez,NE,2022-10-17 05:47:28,19.7500,10.2500,71.56,35,0,9.89
4,Chaman,PK,2022-10-17 05:47:45,30.9236,66.4512,76.42,8,0,5.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# --- store into variable named hotel_df ---

hotel_df = ideal_cities[["City", "Country", "Lat", "Lng"]].copy()

# --- Add a "Hotel Name" column to the DataFrame with null values ---
hotel_df["Hotel Name"] = np.nan
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Makakilo City,US,21.3469,-158.0858,NaN
1,Pahrump,US,36.2083,-115.9839,NaN
2,Antalaha,MG,-14.9003,50.2788,NaN
3,Agadez,NE,19.7500,10.2500,NaN
4,Chaman,PK,30.9236,66.4512,NaN


In [6]:
# --- set up parameters for querying Google Places API ---
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# --- set up a parameters dictionary ---
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# --- define base url ---
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


In [7]:
# --- use a for loop to go through each city in the dataframe and make an API call ---

for index, row in hotel_df.iterrows():
    
    # --- get the coordinates and store in params dictionary ---
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # --- call the API and get hotel details for each city ---
    hotel_data = requests.get(base_url, params).json()
    
    # --- use exception handling to save the first hotel name for each city ---
    
    try:
        hotel_name = hotel_data["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"City '{row['City']}' found! The nearest hotel is {hotel_name}")
        print("-------------------------------------------------------------------------")
              
    except:
        print(f"Could not find information for city '{row['City']}'... skipping.")    
        print("-------------------------------------------------------------------------")
        

City 'Makakilo City' found! The nearest hotel is Hampton Inn & Suites Oahu/Kapolei
-------------------------------------------------------------------------
City 'Pahrump' found! The nearest hotel is Holiday Inn Express & Suites Pahrump, an IHG Hotel
-------------------------------------------------------------------------
City 'Antalaha' found! The nearest hotel is Villa-Malaza
-------------------------------------------------------------------------
Could not find information for city 'Agadez'... skipping.
-------------------------------------------------------------------------
City 'Chaman' found! The nearest hotel is ترکستان باربی کیو ہوٹل
-------------------------------------------------------------------------


In [8]:
# --- remove null rows from dataframe ---
hotel_df.dropna(how="any",inplace=True)
hotel_df.reset_index(drop=True, inplace=True)

# --- display final dataframe ---
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Makakilo City,US,21.3469,-158.0858,Hampton Inn & Suites Oahu/Kapolei
1,Pahrump,US,36.2083,-115.9839,"Holiday Inn Express & Suites Pahrump, an IHG H..."
2,Antalaha,MG,-14.9003,50.2788,Villa-Malaza
3,Chaman,PK,30.9236,66.4512,ترکستان باربی کیو ہوٹل


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# --- add marker layer on top of heat map ---
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='400px', padding='1px', width='100%'))